# Introduction


In [1]:
import numpy as np
import pandas as pd
import pandapower as pp
import power_grid_model as pgm
from generate_fictional_dataset import generate_fictional_grid, generate_time_series
import time
from pathlib import Path
import tempfile

output_dir = Path(tempfile.gettempdir()) / "pandapower_time_series"
output_dir.mkdir(exist_ok=True)


In [2]:
# summary
summary_df = pd.DataFrame(
    np.zeros(shape=(8, 3), dtype=np.float64), 
    columns=['PGM Linear', 'PGM NR', 'PandaPower NR'],
    index=[
        'Symmetric calculation with solver initialization',
        'Symmetric calculation without solver initialization',
        'Asymmetric calculation with solver initialization',
        'Asymmetric calculation without solver initialization',
        'Time series symmetric calculation',
        'Time series asymmetric calculation',
        'N-1 symmetric calculation',
        'N-1 asymmetric calculation',
    ]
)

In [3]:
# fictional grid parameters

n_node_per_feeder = 10
n_feeder = 100

cable_length_km_min = 0.8
cable_length_km_max = 1.2
load_p_w_max = 0.4e6 * 0.8
load_p_w_min = 0.4e6 * 1.2
pf = 0.95

load_scaling_min = 0.5
load_scaling_max = 1.5
n_step = 100
n_node = n_node_per_feeder * n_feeder + 1
n_line = n_node_per_feeder * n_feeder
n_load = n_node_per_feeder * n_feeder

# Pre-cache Library


In [4]:
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=3,
    n_feeder=2,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf
)

pp.runpp(fictional_dataset['pp_net'], algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
pgm_model = pgm.PowerGridModel(fictional_dataset['pgm_dataset'])
pgm_result = pgm_model.calculate_power_flow()

# Single Calculation

In [5]:
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=n_node_per_feeder,
    n_feeder=n_feeder,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf
)

pp_net = fictional_dataset['pp_net']
pgm_dataset = fictional_dataset['pgm_dataset']


## Symmetric

In [6]:
# first calculation with solver initialization
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(calculation_method='linear')
end = time.time()
summary_df.loc['Symmetric calculation with solver initialization', 'PGM Linear'] = end - start

# second calculation with existing solver
start = time.time()
pgm_result = pgm_model.calculate_power_flow(calculation_method='linear')
end = time.time()
summary_df.loc['Symmetric calculation without solver initialization', 'PGM Linear'] = end - start

In [7]:
# first calculation with solver initialization
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow()
end = time.time()
summary_df.loc['Symmetric calculation with solver initialization', 'PGM NR'] = end - start

# second calculation with existing solver
start = time.time()
pgm_result = pgm_model.calculate_power_flow()
end = time.time()
summary_df.loc['Symmetric calculation without solver initialization', 'PGM NR'] = end - start


In [8]:
# first calculation with solver initialization
start = time.time()
pp.runpp(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
end = time.time()
summary_df.loc['Symmetric calculation with solver initialization', 'PandaPower NR'] = end - start

# second calculation with existing solver
start = time.time()
pp.runpp(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
end = time.time()
summary_df.loc['Symmetric calculation without solver initialization', 'PandaPower NR'] = end - start

In [9]:
np.abs(pp_net.res_bus['vm_pu'] - pgm_result['node']['u_pu']).max()

2.3988731663537877e-05

In [10]:
np.abs(pp_net.res_line['loading_percent'] * 1e-2 - pgm_result['line']['loading']).max()

0.0002488719126692851

## Asymmetric

In [11]:
# first calculation with solver initialization
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False, calculation_method='linear')
end = time.time()
summary_df.loc['Asymmetric calculation with solver initialization', 'PGM Linear'] = end - start

# second calculation with existing solver
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False, calculation_method='linear')
end = time.time()
summary_df.loc['Asymmetric calculation without solver initialization', 'PGM Linear'] = end - start

In [12]:
# first calculation with solver initialization
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False)
end = time.time()
summary_df.loc['Asymmetric calculation with solver initialization', 'PGM NR'] = end - start

# second calculation with existing solver
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False)
end = time.time()
summary_df.loc['Asymmetric calculation without solver initialization', 'PGM NR'] = end - start

In [13]:
pp_net.load[['p_mw', 'q_mvar']] = 0.0


# first calculation with solver initialization
start = time.time()
pp.runpp_3ph(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
end = time.time()
summary_df.loc['Asymmetric calculation with solver initialization', 'PandaPower NR'] = end - start

# second calculation with existing solver
start = time.time()
pp.runpp_3ph(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
end = time.time()
summary_df.loc['Asymmetric calculation without solver initialization', 'PandaPower NR'] = end - start


In [14]:
np.abs(pp_net.res_bus_3ph[['vm_a_pu', 'vm_b_pu', 'vm_c_pu']].to_numpy() - pgm_result['node']['u_pu']).max()

5.79704284575655e-10

In [15]:
np.abs(pp_net.res_line_3ph['loading_percent'] * 1e-2 - pgm_result['line']['loading']).max()

8.192209383084403e-09

# Time Series Calculation

In [16]:
time_series_dataset = generate_time_series(
    fictional_dataset,
    n_step=n_step,
    load_scaling_min=load_scaling_min,
    load_scaling_max=load_scaling_max
)

pgm_update_dataset = time_series_dataset['pgm_update_dataset']
pp_dataset = time_series_dataset['pp_dataset']
time_steps = np.arange(n_step)

## Symmetric

In [17]:
start = time.time()
pgm_result = pgm_model.calculate_power_flow(calculation_method='linear', update_data=pgm_update_dataset)
end = time.time()
summary_df.loc['Time series symmetric calculation', 'PGM Linear'] = end - start

In [18]:
start = time.time()
pgm_result = pgm_model.calculate_power_flow(update_data=pgm_update_dataset)
end = time.time()
summary_df.loc['Time series symmetric calculation', 'PGM NR'] = end - start

In [19]:
_ = pp.control.ConstControl(
    pp_net, element='load', element_index=pp_net.load.index,
    variable='p_mw', data_source=pp_dataset['p_mw'], profile_name=pp_net.load.index)
_ = pp.control.ConstControl(
    pp_net, element='load', element_index=pp_net.load.index,
    variable='q_mvar', data_source=pp_dataset['q_mvar'], profile_name=pp_net.load.index)

_ = pp.timeseries.OutputWriter(
    pp_net, output_path=output_dir, output_file_type=".csv", csv_separator=',',
    log_variables=[
        ('res_bus', 'vm_pu'),
        ('res_line', 'loading_percent'),
    ]
)

start = time.time()
pp.timeseries.run_timeseries(
    pp_net, run=pp.runpp, time_steps=time_steps,
    calculate_voltage_angles=True, distributed_slack=True
)
end = time.time()
summary_df.loc['Time series symmetric calculation', 'PandaPower NR'] = end - start

C:\Users\EBA00350\Miniconda3\envs\dnr\lib\site-packages\pandapower\control\util\auxiliary.py:130: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  idx &= net.controller.index[net.controller[df_key] == parameters[df_key]]
recycle is set to True, but internal variables are missing


Progress: |██████████████████████████████████████████████████| 100.0% Complete



In [20]:
pp_u_pu = pd.read_csv(output_dir / 'res_bus' / 'vm_pu.csv', index_col=0).to_numpy()
np.abs(pp_u_pu - pgm_result['node']['u_pu']).max()

2.4175629728562242e-05

In [21]:
pp_loading = pd.read_csv(output_dir / 'res_line' / 'loading_percent.csv', index_col=0).to_numpy() * 1e-2
np.abs(pp_loading - pgm_result['line']['loading']).max()


0.00024974965605661925

## Asymmetric

In [22]:
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False, calculation_method='linear', update_data=pgm_update_dataset)
end = time.time()
summary_df.loc['Time series asymmetric calculation', 'PGM Linear'] = end - start

In [23]:
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False, update_data=pgm_update_dataset)
end = time.time()
summary_df.loc['Time series asymmetric calculation', 'PGM NR'] = end - start

In [24]:
# set symmetric load to zero
pp_net.load[['p_mw', 'q_mvar']] = 0.0

# remove old controller and writer
pp_net.controller.drop(pp_net.controller.index, inplace=True)
del pp_net.output_writer

# define new controller
for x, y in zip(['p', 'q'], ['mw', 'mvar']):
    for p in ['a', 'b', 'c']:
        name = f'{x}_{p}_{y}'
        _ = pp.control.ConstControl(
            pp_net, element='asymmetric_load', 
            element_index=pp_net.load.index, variable=name, 
            data_source=pp_dataset[name], profile_name=pp_net.load.index)

ow = pp.timeseries.OutputWriter(
    pp_net, output_path=output_dir, output_file_type=".csv", csv_separator=',')
ow.log_variable('res_bus_3ph', 'vm_a_pu', index=pp_net.bus.index)   
ow.log_variable('res_bus_3ph', 'vm_b_pu', index=pp_net.bus.index)  
ow.log_variable('res_bus_3ph', 'vm_c_pu', index=pp_net.bus.index)  
ow.log_variable('res_line_3ph', 'loading_percent', index=pp_net.line.index)  


# run
start = time.time()
pp.timeseries.run_timeseries(
    pp_net, run=pp.runpp_3ph, time_steps=time_steps,
    calculate_voltage_angles=True, distributed_slack=True
)
end = time.time()
summary_df.loc['Time series asymmetric calculation', 'PandaPower NR'] = end - start

Progress: |--------------------------------------------------| 1.0% Complete

C:\Users\EBA00350\Miniconda3\envs\dnr\lib\site-packages\pandapower\control\util\auxiliary.py:130: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  idx &= net.controller.index[net.controller[df_key] == parameters[df_key]]


Progress: |██████████████████████████████████████████████████| 100.0% Complete



In [25]:
pp_u_pu = []
for p in ['a', 'b', 'c']:
    pp_u_pu.append(pd.read_csv(output_dir / 'res_bus_3ph' / f'vm_{p}_pu.csv', index_col=0).to_numpy())
pp_u_pu = np.stack(pp_u_pu, axis=-1)
np.abs(pp_u_pu - pgm_result['node']['u_pu']).max()

4.927074304106327e-10

In [26]:
pp_loading = pd.read_csv(output_dir / 'res_line_3ph' / 'loading_percent.csv', index_col=0).to_numpy() * 1e-2
np.abs(pp_loading - pgm_result['line']['loading']).max()

2.4359980699273365e-09

# Summary of Performace

In [27]:
summary_df

,PGM Linear,PGM NR,PandaPower NR
Symmetric calculation with solver initialization,0.002994,0.005024,0.031014
Symmetric calculation without solver initialization,0.002002,0.005025,0.019004
Asymmetric calculation with solver initialization,0.006992,0.025000,0.154083
Asymmetric calculation without solver initialization,0.004000,0.026001,0.132935
Time series symmetric calculation,0.122367,0.459282,2.266740
Time series asymmetric calculation,0.317986,2.114163,17.311900
N-1 symmetric calculation,0.000000,0.000000,0.000000
N-1 asymmetric calculation,0.000000,0.000000,0.000000


In [28]:
pp_net

This pandapower network includes the following parameter tables:
   - bus (1001 elements)
   - load (1000 elements)
   - asymmetric_load (1000 elements)
   - ext_grid (1 element)
   - line (1000 elements)
   - controller (6 elements)
   - output_writer (1 element)
 and the following results tables:
   - res_bus (1001 elements)
   - res_line (1000 elements)
   - res_ext_grid (1 element)
   - res_load (1000 elements)
   - res_bus_3ph (1001 elements)
   - res_line_3ph (1000 elements)
   - res_ext_grid_3ph (1 element)
   - res_load_3ph (1000 elements)
   - res_asymmetric_load_3ph (1000 elements)